In [1]:
from flask import Flask, render_template
from flask_cors import CORS

import json

In [2]:
app = Flask(__name__)

CORS(app)

In [3]:
# from flask_mysqldb import MySQL
# from flask_pymongo import PyMongo

# app.config["MYSQL_HOST"] = "localhost"
# app.config["MYSQL_USER"] = "root"
# app.config["MYSQL_PASSWORD"] = "password"
# app.config["MYSQL_DB"] = "testdb"
# mysql = MySQL(app)

# app.config["MONGO_URI"] = "mongodb://localhost:27017/testdb"
# mongo = PyMongo(app)


# def sql_data():
#     with app.app_context():
#         cur = mysql.connection.cursor()
#         cur.execute("""SELECT * FROM test""")
#         rv = cur.fetchall()
#         cur.close()
#         return rv

# s_data = []
# for i in sql_data():
#     s_data.append(i)
#     print(i)


# def mongo_data():
#     data = mongo.db.test.find()
#     return data

# m_data = []
# for i in mongo_data():
#     m_data.append(i)
#     print(i)

In [4]:
col_keys = [
    'id',
    'name',
    'screen_name',
    'location',
    'url',
    'description',
    'verified',
    'followers_count',
    'friends_count',
    'listed_count',
    'favourites_count',
    'statuses_count',
    'created_at'
]


def make_dict(item):
    kv = {}
    for i in range(len(col_keys)):
        kv[col_keys[i]] = item[i]
        
    return kv
    
def make_dict_list(lst):
    res = []
    
    for item in lst:
        res.append(make_dict(item))  
        
    return res

def extract_val(key, lst):
    res = []
    for item in lst:
        res.append(item[key])
    return res

In [5]:
import mysql.connector as msql

def getUserByWord(word):
    conn = msql.connect(host='localhost', database='TwitterDB', user='root', password='password')
    cursor = conn.cursor()
    query = f"SELECT * FROM users_data WHERE name LIKE '%{word}%' or screen_name LIKE '%{word}%'"
    cursor.execute(query)
    a=cursor.fetchall()
    cursor.close()
    conn.close()
    
    return make_dict_list(a)

def getUserByID(word):
    conn = msql.connect(host='localhost', database='TwitterDB', user='root', password='password')
    cursor = conn.cursor()
    query = f"SELECT * FROM users_data WHERE id = '{word}'"
    cursor.execute(query)
    a=cursor.fetchall()
    cursor.close()
    conn.close()
    return make_dict(a[0]) if len(a)>0 else {}

In [6]:
from pymongo import MongoClient

drop_field = {'_id': 0}

def getTweetbyID(word):
    client = MongoClient()
    db = client["TwitterDB"]
    collection = db["Tweets"]

    query = {"id": word}
    result = collection.find(query, drop_field)
    a=[]
    for i in result:
        a.append(i)
        
    return a[0] if len(a)>0 else {}

def getTweetsbyString(word):
    client = MongoClient()
    db = client["TwitterDB"]
    collection = db["Tweets"]

    query = {"text": {"$regex": word, "$options": "i"} }
    result = collection.find(query, drop_field)
    a=[]
    for i in result:
        a.append(i)
        
    return a

def getTweetsbyHashtag(word):
    client = MongoClient()
    db = client["TwitterDB"]
    collection = db["Tweets"]

    query = {"hashtags": {"$regex": f"{word}", "$options": "i"}}
    result = collection.find(query, drop_field)
    a=[]
    for i in result:
        a.append(i)
        
    return a

def getTweetsbyUserIDsList(lst):
    client = MongoClient()
    db = client["TwitterDB"]
    collection = db["Tweets"]

    query = {"user.id": {"$in": lst}}
    result = collection.find(query, drop_field)
    a=[]
    for i in result:
        a.append(i)
        
    return a

def getTweetsbyUserID(word):
    client = MongoClient()
    db = client["TwitterDB"]
    collection = db["Tweets"]

    query = {"user.id": word}
    result = collection.find(query, drop_field)
    a=[]
    for i in result:
        a.append(i)
        
    return a

def getRetweetsbyUserID(word):
    client = MongoClient()
    db = client["TwitterDB"]
    collection = db["Retweets"]

    query = {"user.id": word}
    result = collection.find(query, drop_field)
    a=[]
    for i in result:
        a.append(i)
        
    return a

def getRetweetsByOriginalTweetID(word):
    client = MongoClient()
    db = client["TwitterDB"]
    collection = db["Retweets"]

    query = {"retweeted_status.id": word}
    result = collection.find(query, drop_field)
    a=[]
    for i in result:
        a.append(i)
        
    return a

In [7]:
@app.route("/")
def index():
    return render_template('index.html')

@app.route("/string/<word>/<stime>/<etime>", methods=["GET"])
def string_tweet(word, stime, etime):
    search_lst = getTweetsbyString(word)
    return json.dumps(search_lst)

@app.route("/hashtag/<word>/<stime>/<etime>", methods=["GET"])
def hashtag_tweet(word, stime, etime):
    search_lst = getTweetsbyHashtag(word)
    return json.dumps(search_lst)

@app.route("/user/<word>/<stime>/<etime>", methods=["GET"])
def user_tweet(word, stime, etime):
    usr_lst = getUserByWord(word)
    id_lst = extract_val('id', usr_lst)
    search_lst = getTweetsbyUserIDsList(id_lst)
    return json.dumps(search_lst)

@app.route("/author/<id>", methods=["GET"])
def get_author(id):
    print(id)
    usr = getUserByID(id)
    usr['tweets'] = getTweetsbyUserID(id)
    usr['retweets'] = getRetweetsbyUserID(id)
    return json.dumps(usr)

@app.route("/retweet/<id>", methods=["GET"])
def get_retweets(id):
    twt = getTweetbyID(id)
    print(twt)
    twt['retweets'] = getRetweetsByOriginalTweetID(id)
    return json.dumps(twt)

In [ ]:
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [26/Apr/2023 17:43:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2023 17:43:26] "GET /static/script.js HTTP/1.1" 304 -
127.0.0.1 - - [26/Apr/2023 17:43:26] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [26/Apr/2023 17:43:27] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [26/Apr/2023 17:44:13] "GET /user/apple/na/na HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2023 17:44:13] "GET /static/verified.png HTTP/1.1" 304 -


1012349457457795072


127.0.0.1 - - [26/Apr/2023 17:55:20] "GET /author/1012349457457795072 HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2023 17:55:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2023 17:55:28] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [26/Apr/2023 17:55:28] "GET /static/script.js HTTP/1.1" 304 -
127.0.0.1 - - [26/Apr/2023 17:55:33] "GET /string/apple/na/na HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2023 17:55:33] "GET /static/verified.png HTTP/1.1" 304 -
127.0.0.1 - - [26/Apr/2023 17:55:39] "GET /retweet/1254023653869543426 HTTP/1.1" 200 -


{'created_at': 1587831912, 'id': '1254023653869543426', 'text': 'Meu celular ya com uma linha verde gigante na tela dps q quebrei e a Apple Store q arruma essas merdas tá fechada por causa do corona', 'hashtags': [], 'user': {'id': '480908581', 'name': 'Gabriela', 'screen_name': 'Sra_Biebeer', 'verified': False, 'friends_count': 346, 'followers_count': 1240, 'description': None, 'created_at': 'Thu Feb 02 02:13:01 +0000 2012'}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 1, 'favorite_count': 0, 'is_quote_status': False}
